In [1]:
from email.policy import default
import json
import os
import pickle
import time
from contextlib import contextmanager
from typing import List, NoReturn, Optional, Tuple, Union
from rank_bm25 import BM25Okapi
import faiss
import numpy as np
import pandas as pd
from datasets import Dataset, concatenate_datasets, load_from_disk
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm.auto import tqdm
from retrieval import *

In [6]:
# Test sparse
# org_dataset = load_from_disk(args.dataset_name)
dataset_name = "../data/train_dataset"
org_dataset = load_from_disk(dataset_name)
full_ds = concatenate_datasets(
    [
        org_dataset["train"].flatten_indices(),
        org_dataset["validation"].flatten_indices(),
    ]
)  # train dev 를 합친 4192 개 질문에 대해 모두 테스트
print("*" * 40, "query dataset", "*" * 40)
print(full_ds)
print(len(org_dataset["train"]),len(org_dataset["validation"]))
from transformers import AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained(args.model_name_or_path, use_fast=False,)
model_name_or_path = "klue/roberta-large"
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=False,)

data_path = "../data"
context_path = "wikipedia_documents.json"
retriever = SparseRetrieval(
    tokenize_fn=tokenizer.tokenize,
    data_path=data_path,
    context_path=context_path,
)

query = "대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?"
retriever.get_sparse_embedding()

with timer("bulk query by exhaustive search"):
    df = retriever.retrieve(full_ds, topk= 40)
    df["correct"] = [original_context in context for original_context,context in zip(df["original_context"],df["context"])]
    print(
        "correct retrieval result by exhaustive search",
        f"{df['correct'].sum()},{len(df)}",
        df["correct"].sum() / len(df),
    )

with timer("single query by exhaustive search"):
    scores, indices = retriever.retrieve(query)

Loading cached processed dataset at ../data/train_dataset/train/cache-9c5e152ac62642fb.arrow
Loading cached processed dataset at ../data/train_dataset/validation/cache-04cef27ebd362e38.arrow


**************************************** query dataset ****************************************
Dataset({
    features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
    num_rows: 4192
})
3952 240
Lengths of unique contexts : 56737
Embedding pickle load.
[query exhaustive search] done in 49.548 s


Sparse retrieval:   0%|          | 0/4192 [00:00<?, ?it/s]

correct retrieval result by exhaustive search 3338,4192 0.7962786259541985
[bulk query by exhaustive search] done in 50.794 s
[transform] done in 0.003 s
[query ex search] done in 0.883 s
[Search query]
 대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은? 

Top-1 passage with score 0.198994
국회에 관해 규정하는 헌법 제4장의 첫 조문이다.

본조에서 말하는 "국권"이란 국가가 갖는 지배권을 포괄적으로 나타내는 국가 권력, 곧 국가의 통치권을 의미한다. 국권은 일반적으로 입법권·행정권·사법권의 3권으로 분류되지만, 그 중에서도 주권자인 국민의 의사를 직접 반영하는 기관으로서 국회를 "최고 기관"으로 규정한 것이다. 다만, 최고 기관이라 해서 타 기관의 감시와 통제를 받지 않는 것은 아니며 권력 분립 원칙에 따라 국회에 대한 행정권, 사법권의 견제를 받는다.

또한 일본 전체 국민을 대표하는 기관을 국회로 규정함으로써, 국회는 일본의 유일한 입법 기관의 지위를 가지고 있다. 일본 제국 헌법 하에서 입법권은 천황의 권한에 속했으며, 제국의회는 천황의 입법 행위를 보좌하는 기관에 불과했다.

여기서 "유일한 입법 기관"의 의미로는 다음과 같은 해석이 있다.
* 국회 중심 입법 원칙 : 국회가 국가의 입법권을 독점한다는 원칙
* 국회 단독 입법 원칙 : 국회의 입법은 다른 기관의 간섭 없이 이루어진다는 원칙

또한 국회의 입법에 벗어나지 않는 범위 내에서 행정 기관은 정령 등의 규칙 제정권을 가지며(헌법 제73조 제6호), 최고재판소는 소송에 관한 절차, 변호사 및 재판소에 관한 내부 규율 및 사법 사무 처리에 관한 사항에 대한 규칙 제정권(헌법 제77조 제1항)을 가진다.
[single query by exhaustive search] done in 0.896 s


In [7]:
df.dtypes

question            object
id                  object
context_id          object
context             object
original_context    object
answers             object
correct               bool
dtype: object

In [14]:
df.head()

,question,id,context_id,context,original_context,answers,correct
0,대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?,mrc-1-000067,"[52322, 1738, 4879, 2269, 22749, 20097, 20771,...","국회에 관해 규정하는 헌법 제4장의 첫 조문이다.\n\n본조에서 말하는 ""국권""이란...",미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...,"{'answer_start': [235], 'text': ['하원']}",True
1,현대적 인사조직관리의 시발점이 된 책은?,mrc-0-004397,"[47823, 47824, 47817, 47815, 47827, 47809, 478...",'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,"{'answer_start': [212], 'text': ['《경영의 실제》']}",True
2,강희제가 1717년에 쓴 글은 누구를 위해 쓰여졌는가?,mrc-1-000362,"[463, 457, 478, 4631, 472, 470, 473, 461, 4626...","강희제는 소년 시절부터 많은 학문을 배웠다. 그중에서도 유학, 즉 성리학을 좋아하였...",강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...,"{'answer_start': [510], 'text': ['백성']}",True
3,11~12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?,mrc-0-001510,"[42192, 29916, 33295, 44184, 30339, 14285, 499...",삼존불비상(三尊佛碑像)은 현재 동국대학교에 있는 것으로 충청남도 공주시 정안면에서 ...,"불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작은 규모로 만든...","{'answer_start': [625], 'text': ['중국']}",True
4,명문이 적힌 유물을 구성하는 그릇의 총 개수는?,mrc-0-000823,"[162, 54365, 15321, 52779, 43524, 15920, 29591...","장음계의 계이름을 떠올리는 것, 또는 기준음을 으뜸음으로 하는 Diatonic Sc...",동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,"{'answer_start': [30], 'text': ['4개']}",True


In [12]:
df["context"]

0       국회에 관해 규정하는 헌법 제4장의 첫 조문이다.\n\n본조에서 말하는 "국권"이란...
1       '근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...
2       강희제는 소년 시절부터 많은 학문을 배웠다. 그중에서도 유학, 즉 성리학을 좋아하였...
3       삼존불비상(三尊佛碑像)은 현재 동국대학교에 있는 것으로 충청남도 공주시 정안면에서 ...
4       장음계의 계이름을 떠올리는 것, 또는 기준음을 으뜸음으로 하는 Diatonic Sc...
                              ...                        
4187    기원전 284년에 이르러 당시 제나라의 왕이었던 제 민왕은 강력한 국력을 믿고 교만...
4188    산대놀이는 한국의 전통 민속놀이이자 무용이다.\n\n공의(公儀)로서 연희되어 오던 ...
4189    1940년 5월 1일 오전 창씨개명에 비협조적이라는 이유로 조선총독부 경무국에서 소...
4190    빌카밤바 지역은 파차쿠티 황제 때 부터 잉카 제국에 속해있던 지역이었다. 스페인 군...
4191    마르크스주의 사회학의 핵심 개념은 역사적 유물론, 생산 양식, 자본과 노동의 관계와...
Name: context, Length: 4192, dtype: object